<a href="https://colab.research.google.com/github/isabelladegen/ComputationalLogic/blob/prolexa-plus/Tests_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Testing Notebook 
Notebook that makes sure Prolexa continues to work


### Installations like demo notebook


In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


### Installing Prolexa
Same error as on the branch on the forked branch https://github.com/simply-logical/ComputationalLogic/

In [2]:
!yes | pip install git+https://github.com/isabelladegen/ComputationalLogic/ -qqq > /dev/null

  ERROR: Failed building wheel for prolexa
  DEPRECATION: prolexa was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Instantiate Prolexa 


In [3]:
from pyswip import Prolog
import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

2021-12-15 11:54:46,564 --------------------------------------------------------------------------------
2021-12-15 11:54:46,565 The model key 'pos' now maps to 'https://huggingface.co/flair/pos-english' on the HuggingFace ModelHub
2021-12-15 11:54:46,566  - The most current version of the model is automatically downloaded from there.
2021-12-15 11:54:46,574  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt)
2021-12-15 11:54:46,576 --------------------------------------------------------------------------------
2021-12-15 11:54:47,361 loading file /root/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63


###  Some Helper functions to make the tests easier to read

In [4]:
def query(input):
  answer = meta.standardised_query(pl, input)[0]['Output']
  if isinstance(answer, str):
    return answer
  return str(answer, 'utf-8')

def test(input, expected_answer):
  print(f'Query: {input}')
  actual_answer = query(input)
  assert actual_answer == expected_answer, 'Expected Answer:  ' + expected_answer + '\n Actual answer: ' + actual_answer
  print(f'Answer: {actual_answer}\n')

def cleanup():
  print(query('forget everything'))
  meta.reset_grammar()
  meta.initialise_prolexa(pl)

### Testing existing functionality

Syntax: `test(input_query, expected_answer)`

Be careful not to add new nouns to Prolexa. The `forget all` method currently deletes all the rules about that noun but not the noun itself

In [5]:
#run before so no unexpected knowledge is haning about
cleanup()

#Tell me about... queries
print("Test 'Tell me about...' queries:\n")
test('Tell me all', 'every human is mortal. peter is human')
test('tell me about unknownnoun', 'I heard you say,  tell me about unknownnoun , could you rephrase that please?')
test('tell me about tweety', 'I know nothing about tweety')
test('tell me about peter', 'peter is human. peter is mortal')

#Is... queries
print("\nTest 'Is...' queries:\n")
test('Is unkownnoun human', 'I heard you say,  Is unkownnoun human , could you rephrase that please?')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Is peter mortal', 'peter is mortal')
test('Is peter human', 'peter is human')
test('Who is human', 'peter is human')
test('Who is a bird', 'Sorry, I don\'t think this is the case')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')

#Explain... queries
print("\nTest 'Is...' queries:\n")
test('Explain why unknownnoun is mortal', 'I heard you say,  Explain why unknownnoun is mortal , could you rephrase that please?')
test('Explain why tweety is a bird','Sorry, I don\'t think this is the case')
test('Explain why peter is mortal', 'peter is human; every human is mortal; therefore peter is mortal')

#Adding new rules and forgeting rules
print("\nTest adding new rules:\n")
test('Peter is mortal', 'I already knew that Peter is mortal')

#add a rule and forget it again
test('Tweety is a bird', 'I will remember that Tweety is a bird')
test('Is tweety a bird', 'tweety is a bird')
test('all birds fly', 'I will remember that all birds fly')
test('Does tweety fly', 'tweety flies')
test('Explain why tweety flies', 'tweety is a bird; every bird is a fly; therefore tweety is a fly') #notice the really odd transaltion <a fly>, the comandline does not do this
test('Tell me all about tweety', 'tweety is a bird. tweety is a fly. tweety is a bird. tweety is a fly') #notice the duplication in answers!!!
test('Forget that tweety is a bird', 'I erased it from my memory')
test('tell me all about tweety', 'I know nothing about tweety')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')


#run after so no unexpected knowledge is haning about for next cells
cleanup()


I am a blank slate
Test 'Tell me about...' queries:

Query: Tell me all
Answer: every human is mortal. peter is human

Query: tell me about unknownnoun
Answer: I heard you say,  tell me about unknownnoun , could you rephrase that please?

Query: tell me about tweety
Answer: I know nothing about tweety

Query: tell me about peter
Answer: peter is human. peter is mortal


Test 'Is...' queries:

Query: Is unkownnoun human
Answer: I heard you say,  Is unkownnoun human , could you rephrase that please?

Query: Is tweety a bird
Answer: Sorry, I don't think this is the case

Query: Is peter mortal
Answer: peter is mortal

Query: Is peter human
Answer: peter is human

Query: Who is human
Answer: peter is human

Query: Who is a bird
Answer: Sorry, I don't think this is the case

Query: Does tweety fly
Answer: Sorry, I don't think this is the case


Test 'Is...' queries:

Query: Explain why unknownnoun is mortal
Answer: I heard you say,  Explain why unknownnoun is mortal , could you rephrase tha

# Testing new functionality


### Default Reasoning

In [ ]:
#Testing new rules
test('Tell me all about opus', 'opus is a bird. opus is a penguin')
test('Tell me all about peep', 'peep is a bird.  peep flies')
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')
test('Is opus a penguin', 'opus is a penguin')
test('Is peep a penguin', 'Sorry, I don\'t think this is the case')

#Tweety is not defined as a bird so
test('Explain why tweety flies', 'Sorry, I don\'t think this is the case')



In [6]:
cleanup()
print(query('tweety flies'))
print(query('tell me all'))
cleanup()
print(query('tell me all'))


I am a blank slate
I will remember that tweety flies
every human is mortal. peter is human. tweety is a fly
I am a blank slate
every human is mortal. peter is human


###  Ideas for new funtionality
 Combine queryies test('Is peter human and mortal', 'peter is human')

 test('Is tweety a bird', 'tweety is a bird')
test('Birds fly', 'I will remember that birds fly')
test('Does tweety fly', 'tweety flies')


In [7]:
#Interesting error about Tweety flies
# print(query('tell me all'))
# print(query('tweety flies'))
# print(query('tell me all'))

Interestingly this problem does not exist if run directly in the backend and it correctly says:
```
prolexa> "Tweety flies".
*** utterance(Tweety flies)
*** rule([(fly(tweety):-true)])
*** answer(I will remember that Tweety flies)
I will remember that Tweety flies
*** utterance(Tell me all)
*** goal(all_rules(_77782))
*** answer(every human is mortal. peter is human. tweety flies)
every human is mortal. peter is human. tweety flies
```

In [8]:
print(query("tweety is a bird"))
print(query('tell me all'))
print(query('Tell me all about tweety'))
print(query('Does tweety fly'))
cleanup()
print(query('Does tweety fly'))
cleanup()
print(query('Explain why tweety flies'))

I will remember that tweety is a bird
every human is mortal. peter is human. tweety is a bird
tweety is a bird. tweety is a bird
Sorry, I don't think this is the case
I am a blank slate
Sorry, I don't think this is the case
I am a blank slate
Sorry, I don't think this is the case


In [9]:
print(query('All birds fly'))
print(query('tell me all about tweety'))
print(query('does tweety fly'))

I will remember that All birds fly
I know nothing about tweety
Sorry, I don't think this is the case


In [10]:
#Prove
test('Prove Peter is human', '')

Query: Prove Peter is human


AssertionError: ignored